<a href="https://colab.research.google.com/github/whorseman/Assignments/blob/main/assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following assignment consists again of a theoretical part (learning portfolio) and a practical part (assignment). The goal is to train a neural model for a recommendation system.

The plan would be that in the first week we will discuss your learnings from the theory part, that means you are relatively free to fill your Learning Portfolio on this topic and in the following week we will discuss your solutions of the practical part.

#Theory part (filling your Learning Portfolio, June 7)

In preparation for the practical part, I ask you to familiarize yourself with the following video sources in the next week:

1) Please watch the following videos:

https://www.youtube.com/watch?v=Fmtorg_dmM0&ab_channel=ritvikmath (not absolutely necessary, only for the overview)

https://course.fast.ai/Lessons/lesson7.html (The second part of the presentation starting with the topic collaborative filtering is mandatory)

Note: The first part of the video mainly contains tips for neural networks to submit a Kaggle Competition. For that, you would have to watch the end of the 6th video to understand this better. But this is not mandatory.

2) Please download the following notebooks and edit it in Google-Colab. Try to answer a few questions that are asked at the end. Take notes and update your Learning Portfolio.

https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook


#Practical part (Assignment, June 14)

Find any data set that can be used for a recommender system and try to train and validate a neural network for it.

For this purpose I ask you to download a data set from the given lists and to use it for your program application.

https://gist.github.com/entaroadun/1653794

https://github.com/caserec/Datasets-for-Recommender-Systems

https://grouplens.org/datasets/movielens/

https://eigentaste.berkeley.edu/dataset/

In [1]:
from fastai.collab import *
from fastai.tabular.all import *
set_seed(42)

import pandas as pd
from google.colab import drive
from google.colab import data_table
import matplotlib.pyplot as plt
data_table.enable_dataframe_formatter()

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_json("/content/drive/MyDrive/DIGO/Digital_Music_5.json", lines=True)

In [3]:
#keeping only the desired columns, reviewer, Amazon ID number and overall rating
d1 = data[['reviewerID','asin','overall']]

#creating a dataloader from the frame
dls = CollabDataLoaders.from_df(d1, item_name='asin', bs=64)

#get recommended embeddings size
embs = get_emb_sz(dls)
embs

[(5542, 200), (3569, 156)]

In [4]:
from torch import arcsin
import requests
import json

# set up the request parameters
def get_name(asin):
  params = {
  'api_key': 'B9C5261506574C32A89D8ADC249660DD',
    'asin': str(asin),
    'type': 'product',
    'amazon_domain': 'amazon.de'
  }

  # make the http GET request and extract the title
  api_result = requests.get('https://api.asindataapi.com/request', params)
  title = api_result.json()['product']['title']
  return(title)


In [35]:
get_name('B000BJS4P8')

'Duets-the Final Chapter'

In [23]:
dls.show_batch()

,reviewerID,asin,overall
0,A1ETANQJNZCP4F,B000002N25,5
1,AO02F8SE42I62,B000BJS4P8,4
2,A39OKUD8SO91G3,B000002BA5,5
3,A2GT65YTNYXBW3,B000001ABG,3
4,A2YTW6PRPUGDDP,B000001DYS,5
5,AZDVOFC2MTIM5,B00000016W,5
6,A2I2A7N3OCPNW4,B007PXNUMU,4
7,A1ZTC02LF6GX1H,B002JUJRTO,5
8,A33693RRRFK55Q,B000002MUW,4
9,A1N5FSCYN4796F,B000002VP3,5


In [5]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range

    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

model = CollabNN(*embs)

In [7]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-2, wd=0.07)

epoch,train_loss,valid_loss,time
0,1.373925,1.572319,00:19
1,1.187276,1.157827,00:19
2,0.908201,0.907626,00:20
3,0.833643,0.834421,00:19
4,0.669320,0.823302,00:19
